In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

import pandas as pd
import tensorflow as tf
import pandas as pd 

from google.colab import files
uploaded = files.upload()

Saving data.csv to data.csv


In [2]:
#Read csv file into a dataframe
import io 
data_df = pd.read_csv(io.BytesIO(uploaded['data.csv']))

In [ ]:
data_df.head()

,Email No.,Prediction,the,to,ect,and,for,of,a,you,...,enhancements,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry
0,Email 1,0,0,0,1,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
1,Email 2,0,8,13,24,6,6,2,102,1,...,0,0,0,0,0,0,0,0,1,0
2,Email 3,0,0,0,1,0,0,0,8,0,...,0,0,0,0,0,0,0,0,0,0
3,Email 4,0,0,5,22,0,5,1,51,2,...,0,0,0,0,0,0,0,0,0,0
4,Email 5,0,7,6,17,1,5,2,57,0,...,0,0,0,0,0,0,0,0,1,0


In [3]:
data_df.nunique()

Email No.         5172
Prediction           2
the                 90
to                  72
ect                 90
                  ... 
infrastructure       4
military             4
allowing             3
ff                  23
dry                  4
Length: 3002, dtype: int64

In [4]:
#Look at the sums for each column (i.e. the number of times the word appears in emails) 
sum_df = data_df.drop(columns=['Email No.'])
sum_df = sum_df.sum().sort_values(ascending=False)
sum_df

e            438561
t            312791
a            287136
o            284970
n            248777
              ...  
padre            21
felipe           21
dorcheus         21
pooling          21
decisions        21
Length: 3001, dtype: int64

In [5]:
#Find all the columns whos sum is less than 50 (i.e. the word appears less than 50 times)
low_cols = [col for col in data_df.columns[1:] if data_df[col].sum() <= 50]
low_cols

['talked',
 'behalf',
 'hplnol',
 'somehow',
 'highly',
 'titles',
 'ftar',
 'nominates',
 'centana',
 'kinsey',
 'goals',
 'acceptance',
 'josey',
 'rather',
 'egmnom',
 'lowest',
 'talking',
 'understood',
 'logos',
 'notified',
 'connor',
 'fontfont',
 'almost',
 'crude',
 'clearance',
 'ftworth',
 'resellers',
 'counterparties',
 'cartwheel',
 'mtbe',
 'split',
 'submitted',
 'baseload',
 'rights',
 'lost',
 'membership',
 'invitation',
 'legislation',
 'guadalupe',
 'zivley',
 'pipelines',
 'equity',
 'dealers',
 'claims',
 'merchant',
 'braband',
 'topica',
 'edmondson',
 'participate',
 'throughout',
 'checked',
 'contents',
 'investments',
 'moving',
 'proposed',
 'deer',
 'enquiries',
 'reduction',
 'evening',
 'concerning',
 'cowboy',
 'minimum',
 'changing',
 'built',
 'strategy',
 'thousands',
 'resulting',
 'policy',
 'positive',
 'quantity',
 'programs',
 'airmail',
 'prepared',
 'austin',
 'uae',
 'jaquet',
 'hplr',
 'weather',
 'terminated',
 'venturatos',
 'cellpadding

In [6]:
#Count how many columns had sums less than 50.
low_count = len(low_cols)
low_count

815

In [7]:
#Drop the low count columns from the df (3002 original columns - 815 dropped = 2187 columns)
new_df = data_df.copy()
for col in new_df.columns:
    if col in low_cols:
        del new_df[col]
new_df.head()

,Email No.,Prediction,the,to,ect,and,for,of,a,you,...,del,trial,ended,fall,hate,plain,jay,valued,lay,ff
0,Email 1,0,0,0,1,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
1,Email 2,0,8,13,24,6,6,2,102,1,...,0,0,0,0,0,0,0,0,0,1
2,Email 3,0,0,0,1,0,0,0,8,0,...,0,0,0,0,0,0,0,0,0,0
3,Email 4,0,0,5,22,0,5,1,51,2,...,0,0,0,0,0,0,0,0,0,0
4,Email 5,0,7,6,17,1,5,2,57,0,...,0,0,0,0,0,0,0,0,0,1


In [8]:
model_df = new_df.drop(columns=['Email No.'])
model_df

,Prediction,the,to,ect,and,for,of,a,you,hou,...,del,trial,ended,fall,hate,plain,jay,valued,lay,ff
0,0,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,8,13,24,6,6,2,102,1,27,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,1,0,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,5,22,0,5,1,51,2,10,...,0,0,0,0,0,0,0,0,0,0
4,0,7,6,17,1,5,2,57,0,9,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5167,0,2,2,2,3,0,0,32,0,0,...,0,0,0,0,0,0,0,0,0,0
5168,0,35,27,11,2,6,5,151,4,3,...,0,0,0,0,0,0,0,0,0,1
5169,1,0,0,1,1,0,0,11,0,0,...,0,0,0,0,0,0,0,0,0,0
5170,1,2,7,1,0,2,1,28,2,0,...,0,0,0,0,0,0,0,0,0,1


In [10]:
model_df = pd.get_dummies(model_df, dtype=float)
model_df.head()

,Prediction,the,to,ect,and,for,of,a,you,hou,...,del,trial,ended,fall,hate,plain,jay,valued,lay,ff
0,0,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,8,13,24,6,6,2,102,1,27,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,1,0,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,5,22,0,5,1,51,2,10,...,0,0,0,0,0,0,0,0,0,0
4,0,7,6,17,1,5,2,57,0,9,...,0,0,0,0,0,0,0,0,0,1


In [11]:
# Split our preprocessed data into our features and target arrays
y = model_df['Prediction'].values
X = model_df.drop('Prediction', axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 1)

Step 1: Fit a logistic regression model by using the training data (X_train and y_train).

In [13]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1)

# Fit the model using training data
classifier.fit(X_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=1)

Step 2: Save the predictions on the testing data labels by using the testing feature data (X_test) and the fitted model.

In [14]:
# Make a prediction using the testing data
test_predictions = classifier.predict(X_test)
test_predictions_df = pd.DataFrame({"Prediction": test_predictions, "Actual": y_test})
test_predictions_df

,Prediction,Actual
0,0,0
1,1,1
2,0,0
3,0,0
4,1,1
...,...,...
1288,0,0
1289,0,0
1290,1,1
1291,1,1


Step 3: Evaluate the model’s performance by doing the following:
Calculate the accuracy score of the model.

Generate a confusion matrix.

Print the classification report.

In [15]:
# Print the balanced_accuracy score of the model
from sklearn.metrics import accuracy_score
test_accuracy = accuracy_score(y_test, test_predictions)
test_accuracy

0.9651972157772621

In [16]:
# Generate a confusion matrix for the model
from sklearn.metrics import confusion_matrix
test_matrix = confusion_matrix(y_test, test_predictions)
test_matrix

array([[889,  20],
       [ 25, 359]])

In [21]:
# Print the classification report for the model
from sklearn.metrics import classification_report
target_names = ['Spam', 'No Spam']
print(classification_report(y_test, test_predictions, target_names=target_names))

              precision    recall  f1-score   support

        Spam       0.97      0.98      0.98       909
     No Spam       0.95      0.93      0.94       384

    accuracy                           0.97      1293
   macro avg       0.96      0.96      0.96      1293
weighted avg       0.97      0.97      0.97      1293

